In [1]:
import sys
from pathlib import Path
import pandas as pd
import tarfile
import urllib

def load_metadata():
  file_path = Path("data/train.csv")
  return pd.read_csv(file_path)
  
metadata = load_metadata()

def extract_eeg():
  eeg_dir = Path("../data/eeg")
  tarball_path = Path("data/eeg.tar.gz")
  if not tarball_path.is_file():
    url = 'https://dl.dropboxusercontent.com/scl/fi/5sina48c4naaxv6uze0fv/eeg.tar.gz?rlkey=r7ec191extynfcm8fy0tsiws5&dl=0'
    urllib.request.urlretrieve(url, tarball_path)
    with tarfile.open(tarball_path) as eeg_tarball:
      eeg_tarball.extractall()
    
extract_eeg()

metadata

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,4144388963,140,604.0,1156825996,140,604.0,1451266906,59489,GRDA,0,0,0,0,3,0
1,2353475448,30,64.0,1002394133,30,64.0,4000072340,5339,LRDA,0,0,0,3,0,0
2,1618328341,9,52.0,900482955,9,52.0,4140697659,20198,GRDA,0,0,0,0,3,0
3,979865826,7,90.0,1626043434,7,90.0,919550440,1069,Other,1,1,4,1,4,5
4,521108392,0,0.0,827447277,0,0.0,1717414556,13134,Other,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2509824693,10,68.0,1005228554,15,330.0,219919562,14386,LPD,0,11,0,1,1,2
996,2882719839,14,50.0,2035369578,14,50.0,4193559045,2641,GPD,5,0,11,0,0,0
997,1322226281,2,18.0,1740512896,2,18.0,1697286566,49448,Other,0,0,0,0,0,3
998,628369060,15,98.0,13143748,17,292.0,1650460145,34998,GPD,0,3,7,0,2,4


In [2]:
import dask.dataframe as dd
from src.utils import compute_signal_hash

channel_order = ['Fp1', 'Fp2',
            'F7', 'F3', 'Fz', 'F4', 'F8', 
            'T3', 'C3', 'Cz', 'C4', 'T4', 
            'T5', 'P3', 'Pz', 'P4', 'T6', 
            'O1', 'O2',
          ]

def load_signals(metadata):
  rows = len(metadata)
  eeg_list = []

  for row in range(0,rows):
    sample = metadata.iloc[row]
    f_name = f'data/eeg/{sample.eeg_id}.parquet'
    eeg = pd.read_parquet(f_name)[channel_order]
    eeg_offset = int(sample.eeg_label_offset_seconds)

    eeg['id'] = str(compute_signal_hash(sample))
    eeg = eeg.set_index('id')

    eeg = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]
    eeg_list.append(eeg)

  return dd.concat(eeg_list)

ddf = load_signals(metadata)


In [3]:
df = ddf.compute()
df

sig_ids = list(set(df.index.to_list()))
sig_ids


['fb7b93120',
 '9dcb91345',
 '38ff8e4a0',
 '3142cc877',
 '157896dd8',
 'b84cf11bb',
 '83abba957',
 '00cac834f',
 '3c3f396da',
 '607988ecd',
 '09f74b84e',
 'e33cfb3dd',
 'a13e59511',
 '0b0c9d79a',
 'a81db975e',
 '677f4b099',
 'f73a7e258',
 '6a6821ead',
 'db76e9927',
 'c8f15b72c',
 '67271fc3c',
 '97d9e5985',
 '3b8d3f30d',
 '1cf3a79de',
 '1cfc4bc05',
 'b9d0e0b9d',
 'ade150e7d',
 '51ce45a7d',
 'd34b35789',
 '4a550c8ca',
 'd880c8ed9',
 'df20a2bdd',
 '9ed82fcc9',
 'b4d3de6d9',
 '332c25743',
 '1de596cf7',
 '5579f813c',
 '544d6e9e0',
 '92e7d62cf',
 '8c9f414a2',
 '8a91cb1ad',
 '01976ec69',
 '679de393f',
 'ef9bc8aca',
 'fb2bfb3db',
 '6c6b2d7e4',
 '54923d74e',
 '1cb65c7fd',
 '545077760',
 '820fbbf05',
 '23cb5d271',
 '2c14ad8bd',
 '29ddc7456',
 'a2e496b6b',
 '824e3c02f',
 '4e3c04ebb',
 'c6987016e',
 '65728fdae',
 'ed0df7a47',
 '8bb17581e',
 '9c2ddd8bb',
 '2ad53794c',
 'aa1941a91',
 '1607b5456',
 '9d000faf8',
 '3e1c8139e',
 '866bd3fec',
 'ffb9a987d',
 'f599e6794',
 '4c718f2f8',
 'b044d1bbc',
 'c8f4

In [4]:
## Apply the wavelet transform
import pywt
from src.preprocessing import wavelet_transform

sig1 = df.loc['554a28223']
max_levels = pywt.dwt_max_level(data_len=10000, filter_len=pywt.Wavelet('coif1').dec_len)

wavelet_transform(sig1, max_levels)


/Users/gagetylee/Documents/Projects/2024_1_project_11/src/preprocessing/preprocessing.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[channel] = cleaned_channel


,Fp1,Fp2,F7,F3,Fz,F4,F8,T3,C3,Cz,C4,T4,T5,P3,Pz,P4,T6,O1,O2
id,,,,,,,,,,,,,,,,,,,
554a28223,-984.929443,-876.999512,-1226.049316,-906.509583,-13.559980,-28.709984,-928.569397,-12.759975,27.379980,7.709996,-23.389975,-19.409975,19.939987,46.519974,36.419968,20.199986,-20.739973,65.659935,7.179996
554a28223,-259.719666,-269.559692,-474.519531,-211.609726,-9.569979,-32.699982,-203.369614,-23.929974,21.799982,11.169996,-26.849974,-24.189976,12.229988,39.339970,30.309969,12.759988,-26.579975,58.479931,0.799998
554a28223,489.940125,365.790100,359.410339,555.340149,-16.749979,-32.959980,450.330109,-24.989971,16.219984,2.659998,-27.649973,-23.129972,11.169990,36.149975,27.909964,11.699989,-25.519979,55.289936,1.059998
554a28223,-666.989624,-620.469727,-893.219421,-586.169739,-20.469976,-31.369980,-662.999695,-23.389973,14.089987,-2.130001,-27.119970,-19.139971,13.559990,35.359978,27.909969,12.489988,-22.329977,55.029934,3.189999
554a28223,-648.109619,-611.159607,-908.639465,-595.479736,-14.359975,-34.289982,-589.629639,-27.649973,11.429989,4.249999,-28.709961,-23.129972,7.709991,29.239981,21.799971,6.909989,-27.119974,49.449944,-1.329999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554a28223,-5065.817871,-3398.749512,-5890.988281,-4519.258789,-19.939960,-39.079956,-4315.358398,-9.299988,-10.099981,-24.719967,-43.069942,-10.369980,3.989998,9.039993,-7.439980,-8.239982,-9.039986,12.759991,-7.709983
554a28223,2149.570068,1264.330322,753.921753,1882.400391,-24.189964,-62.739956,1797.870117,-39.339981,-27.119982,-17.279976,-67.519943,-38.549976,-25.249994,-16.750004,-30.569981,-35.889977,-37.219982,-14.360005,-36.149979
554a28223,3339.200195,2796.620361,2563.481445,3421.879883,-44.929962,-52.369949,3363.389893,-20.469984,-31.369984,-49.709965,-55.559948,-18.609983,-5.849998,-5.320005,-21.269983,-18.339981,-16.479984,-0.800007,-16.479980


In [5]:
sig1 = df.loc['554a28223']

In [6]:
## MNE setup
import mne
import numpy as np

mne_info = mne.create_info(ch_names=sig1.columns.tolist(), sfreq=200, ch_types='eeg')
mne_info.set_montage('standard_1020')
    
sig1_data = np.array(sig1.transpose())
sig1_data = np.nan_to_num(sig1_data)
    
raw = mne.io.RawArray(sig1_data, mne_info)
raw.apply_function(lambda x: x / 20e6, picks='eeg')

Creating RawArray with float64 data, n_channels=19, n_times=10000
    Range : 0 ... 9999 =      0.000 ...    49.995 secs
Ready.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,22 points
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,200.00 Hz
Highpass,0.00 Hz
Lowpass,100.00 Hz


In [7]:
# Apply filters
from src.preprocessing import notch_filter, bp_filter, standardize

l_freq = 1.0
h_freq = 70.0

df = notch_filter(df, 60)
df = bp_filter(df, 1.0, 70)
df = standardize(df)
df

Creating RawArray with float64 data, n_channels=19, n_times=10000000
    Range : 0 ... 9999999 =      0.000 ... 49999.995 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1321 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.3s


Creating RawArray with float64 data, n_channels=19, n_times=10000000
    Range : 0 ... 9999999 =      0.000 ... 49999.995 secs
Ready.


KeyboardInterrupt: 

In [ ]:
# Extracting top 3 channels based on variance for all samples
# 1000 samples computation duration = approx. 10 minutes
from src.feature_extraction import calculate_all_samples

top_channels_df = calculate_all_samples(df, sig_ids, 10) # 10 samples for testing
top_channels_df # NEED TO EXTRACT FEATURES FROM CHANNELS IN THIS DATA STRUCTURE

,0,1,2
ec2a8e98d,F7,Pz,Fp1
e7ce7ea53,F3,Pz,P3
110270772,T6,F8,T4
4a8ace4a7,P3,T3,C3
f01a73f05,Fp1,F3,T5
...,...,...,...
bc498a548,NaN,NaN,NaN
da0bd345b,NaN,NaN,NaN
4f2d7b60b,NaN,NaN,NaN
d5d2a9f36,NaN,NaN,NaN


In [ ]:
# Relevant code for testing purposes

# print(np.var(df['Fp1'], axis=0)) #variance for one col/channel
# fpl = df['Pz'].fillna(0).to_numpy() #converting to numby array for easier computation
#print(np.var(df['Fp1'], axis=0)) #variance for one col/channel

# one sample and their channels
# sig1 = df.loc[['521108392']]
# sig1
# sig1['Fp1'] # one sample and single channel
# print(np.var(fpl, axis=0))

# variance for one channel(Fp1) in one signal(4144388963)
# np.var(sig1['F7'].to_numpy())

# File created to test the correctness of extracted values using MATLAB
# Save Fp1 channel data into a MATLAB file
# import scipy.io
# scipy.io.savemat('Fp1_data.mat', {'Fp1_data': sig1['Fp1']})